In [4]:
import pandas as pd

filtered_data = pd.read_csv('./processed.csv')

/var/folders/wh/srwjqw_j5gsbl1y7xdb9jc900000gn/T/ipykernel_36251/1112237029.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_data = pd.read_csv('./processed.csv')


In [5]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and targets (y)
X = filtered_data.drop(columns=['StartHesitation', 'Turn', 'Walking'])
y = filtered_data[['StartHesitation', 
                   'Turn', 
                   'Walking',
                  ]]

# Split the data into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Combine the features and targets back into DataFrames for AutoGluon
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)


del filtered_data

In [6]:
from autogluon.tabular import TabularDataset, TabularPredictor
import gc

# Assuming 'train_data' and 'test_data' are DataFrames created from the previous response
# Convert the data to AutoGluon's TabularDataset format
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)

# Define the target columns
labels = ['StartHesitation', 'Turn', 'Walking']

# Initialize an empty dictionary to store the predictors
predictors = {}

# Train a separate regression model for each target event type
for label in labels:
    print(f"Training model for {label}...")
    predictor = TabularPredictor(label=label, 
                                 problem_type='regression', 
                                 eval_metric='mean_absolute_error') # regression with R^2 as the evaluation metric
    predictor.fit(train_data, 
                #   num_gpus=2, 
                  excluded_model_types=['LightGBMLarge'], 
#                   time_limit=600, 
#                   num_bag_sets = 2, 
                #   hyperparameters = 'light',
                  )

    predictors[label] = predictor




No path specified. Models will be saved in: "AutogluonModels/ag-20230501_180317/"


Training model for StartHesitation...


	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230501_180317/"
AutoGluon Version:  0.7.0
Python Version:     3.10.4
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 22.1.0: Sun Oct  9 20:14:30 PDT 2022; root:xnu-8792.41.9~2/RELEASE_ARM64_T8103
Train Data Rows:    6103075
Train Data Columns: 7
Label Column: StartHesitation
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    890.75 MB
	Train Data (Original)  Memory Usage: 565.44 MB (63.5% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 

ValueError: AutoGluon did not successfully train any models

In [ ]:
# Make predictions on the test data for each target event type
predictions = {}
for label in labels:
    print(f"Predicting probabilities for {label}...")
    predictor = predictors[label]
    predictions[label] = predictor.predict(test_data.drop(columns=labels))
    

# Combine the predictions into a single DataFrame
predictions_df = pd.DataFrame(predictions)
print(predictions_df)